# Анализ работы модели предсказания качества воздуха

В работе используются:
- модель с [Kaggle](https://www.kaggle.com/code/hamedetezadi/air-quality-prediction), в формате [ноутбука](https://colab.research.google.com/drive/14Tlqrmg76i7S33Gc5P5iFvjh7Ml2dXxB?usp=sharing)
- датасет  «СОСТОЯНИЕ ЗАГРЯЗНЕНИЯ АТМОСФЕРЫ В
 ГОРОДАХ РОССИИ C 2007 ГОДА» - приложено в файлах ноута


# Скачивание датасета

In [19]:
import os
import kagglehub

dataset_name = "shrutibhargava94/india-air-quality-data"
dataset_files_path = kagglehub.dataset_download(dataset_name)

print("Path to dataset files:", dataset_files_path)

# перекладываем в проект то, что скачали в нормальной кодировке (data.csv)
for filename in os.listdir(dataset_files_path):
    source_file = os.path.join(dataset_files_path, filename)
    if os.path.isfile(source_file):
        destination_file = os.path.join(os.getcwd(), filename)
        with open(source_file, 'rb') as src:
            content = src.read()
            decoded_content = content.decode('utf-8', errors='ignore')

        with open(destination_file, 'w') as dst:
            dst.write(decoded_content)

Path to dataset files: C:\Users\Ingvar\.cache\kagglehub\datasets\shrutibhargava94\india-air-quality-data\versions\1


## Запускаем модель

In [27]:
import joblib
import pandas as pd

# Загрузка обученной модели
model_path = 'Air_Quality_Prediction.joblib'  # Укажите путь к вашей модели
try:
    model = joblib.load(model_path)
    print("Модель успешно загружена.")
except FileNotFoundError:
    print(f"Ошибка: модель не найдена по пути {model_path}")
    model = None

# Применение модели для прогнозов (если модель загружена)
if model:
    try:
        # TODO то, что выкачивается с каггла (india-air-quality-data) модель не ест
        data = pd.read_csv("data.csv")
        predictions = model.predict(data)
        data['Predictions'] = predictions
        print("Прогнозы успешно выполнены.")

        # Сохранение данных с прогнозами
        predictions_file_path = 'predictions_air_quality.csv'
        data.to_csv(predictions_file_path, index=False)
        print(f"Данные с прогнозами сохранены в файл {predictions_file_path}")
    except Exception as e:
        print(f"Ошибка при выполнении прогнозов: {e}")

C:\Users\Ingvar\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.6.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Модель успешно загружена.
Ошибка при выполнении прогнозов: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- agency
- date
- location
- location_monitoring_station
- no2
- ...
Feature names seen at fit time, yet now missing:
- Noi
- Rpi
- SOi
- SPMi



C:\Users\Ingvar\AppData\Local\Temp\ipykernel_17360\4121270145.py:17: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data.csv")


In [24]:
expected_features = model.feature_names_in_
print("Ожидаемые моделью признаки:", expected_features)
print("Текущие признаки данных:", data.columns.tolist())


Ожидаемые моделью признаки: ['SOi' 'Noi' 'Rpi' 'SPMi']
Текущие признаки данных: ['stn_code', 'sampling_date', 'state', 'location', 'agency', 'type', 'so2', 'no2', 'rspm', 'spm', 'location_monitoring_station', 'pm2_5', 'date']


In [ ]:
import pandas as pd
import numpy as np
import joblib
import re

# Загрузка данных
file_path = 'data_air_cities_100_v20231129.csv'
dataset_files_path = pd.read_csv(file_path, sep=';', engine='python')

# Очистка числовых данных от текста
def extract_first_number(value):
    if isinstance(value, str):
        match = re.search(r'\d+(\.\d+)?', value)
        return float(match.group()) if match else np.nan
    return value

numeric_columns = ['air_solid_emissions', 'air_so_emissions', 'air_no_emissions', 'air_co_emissions', 'air_population', 'air_repeatability']
for col in numeric_columns:
    dataset_files_path[col] = dataset_files_path[col].apply(extract_first_number)

# Обработка пропущенных значений
categorical_columns = ['region', 'city', 'air_general_level', 'air_standard_index']

# Заполнение числовых данных средним значением
dataset_files_path[numeric_columns] = dataset_files_path[numeric_columns].fillna(dataset_files_path[numeric_columns].mean())

# Заполнение категориальных данных модой
for col in categorical_columns:
    dataset_files_path[col] = dataset_files_path[col].fillna(dataset_files_path[col].mode()[0])

# Нормализация числовых данных (мин-макс нормализация)
dataset_files_path[numeric_columns] = (dataset_files_path[numeric_columns] - dataset_files_path[numeric_columns].min()) / (
        dataset_files_path[numeric_columns].max() - dataset_files_path[numeric_columns].min()
)

# Кодирование категориальных данных
data_encoded = pd.get_dummies(dataset_files_path, columns=categorical_columns, drop_first=True)

# Загрузка модели
model_path = 'Air_Quality_Prediction.joblib'
model = joblib.load(model_path)

# Подготовка данных для модели
expected_features = model.feature_names_in_
missing_features = set(expected_features) - set(data_encoded.columns)

# Добавляем отсутствующие признаки, заполняя их нулями
for feature in missing_features:
    data_encoded[feature] = 0

# Оставляем только нужные признаки для модели
data_final = data_encoded[list(expected_features)]

# Применение модели для прогнозов
predictions = model.predict(data_final)
data_final['Predictions'] = predictions

# Сохранение обработанных данных
processed_file_path = 'processed_data_air_quality.csv'
data_final.to_csv(processed_file_path, index=False)
print(f"Обработанные данные сохранены в {processed_file_path}")

# Сохранение прогнозов
predictions_file_path = 'predictions_air_quality.csv'
data_final.to_csv(predictions_file_path, index=False)
print(f"Прогнозы сохранены в {predictions_file_path}")


Обработанные данные сохранены в processed_data_air_quality.csv
Прогнозы сохранены в predictions_air_quality.csv


<ipython-input-13-5b05d44382f1>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final['Predictions'] = predictions
